In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

from hepattn.experiments.clic.performance.performance import Performance, PerformanceConfig
from hepattn.experiments.clic.performance.plot_helper import PlotHelper

In [ ]:
config_dict = {
    "truth_path": "/storage/agrp/dmitrykl/hgpf/hepformer/data/nilo/test_clic_common_raw.root",
    "networks": [
        {
            "name": "mpflow",
            "path": "/storage/agrp/dmitrykl/hgpf/hepattn/src/hepattn/experiments/clic/logs/CLIC_Pflow_FullDiceFocFix_20250613-T142512/ckpts/epoch=159-val_loss=3.51694__test__common.root",
            "network_type": "mpflow_proxy",
            "ind_threshold": 0.65,
        },
        {
            "name": "hgpflow",
            "path": "/storage/agrp/nilotpal/HGPFlow_v2/experiments/hgpflow_v2/clicxminixbig1x2xs2xxxds7kirm1yo/inference/ee_qq_new/pred_test_p8_ee_qq_ecm380_20k_seg_bw0.3_nocut_merged.root",
            "network_type": "hgpflow_proxy",
            "ind_threshold": 0.65,
        },
        {
            "name": "mlpf",
            "path": "/srv01/agrp/dmitrykl/projects/mlpf/particleflow/experiments/pyg-clic-my_20250726_155449_087474/test/preds_common_checkpoint-21-2.657597/batch_size",
            "network_type": "mlpf",
        },
    ],
}

In [ ]:
config = PerformanceConfig.from_dict(config_dict)

In [ ]:
perf_obj = Performance(config)

In [ ]:
perf_obj.reorder_and_find_intersection()

In [ ]:
perf_obj.compute_jets(n_procs=20)

In [ ]:
perf_obj.hung_match_jets()
perf_obj.compute_event_features()
perf_obj.compute_jet_res_features(dr_cut=0.1, leading_n_jets=2, pt_min=10)

In [ ]:
from collections import defaultdict


def default_style_dict():
    return {
        "histtype": "step",
        "linewidth": 1,
    }


style_dict = defaultdict(default_style_dict)
style_dict["pandora"] = {
    "color": "gray",
    "alpha": 0.5,
    "histtype": "stepfilled",
}

In [ ]:
plot_helper = PlotHelper(perf_obj, style_dict=style_dict)

In [ ]:
fig = plot_helper.plot_jet_residuals()

In [ ]:
fig = plot_helper.plot_evt_res()

In [ ]:
pt_bins = np.array([0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200])

In [ ]:
fig = plot_helper.plot_jet_res_boxplot(bins=pt_bins)

In [ ]:
fig = plot_helper.plot_jet_response(pt_bins=pt_bins, use_energy=True)

In [ ]:
perf_obj.hung_match_particles(flatten=True, return_unmatched=True)

In [ ]:
qs = {"Charged": {"pt": 90, "eta": 80, "phi": 80}, "Neutral": {"pt": 90, "eta": 80, "phi": 80}}
fig = plot_helper.plot_residuals(pt_relative=True, log_y=True, qs=qs)

In [ ]:
qs = {"Neutral hadron": {"pt": 98, "eta": 75, "phi": 75}, "Photon": {"pt": 99, "eta": 90, "phi": 90}}
fig = plot_helper.plot_residuals_neutrals(pt_relative=True, log_y=True, qs=qs)

In [ ]:
eff_fr_colors = {
    "mpflow": {
        "neut had": "mediumseagreen",
        "photon": "tomato",
    },
    "hgpflow": {
        "neut had": "steelblue",
        "photon": "darkorange",
    },
    "pandora": {
        "neut had": "dodgerblue",
        "photon": "goldenrod",
    },
    "mlpf": {
        "neut had": "crimson",
        "photon": "darkviolet",
    },
}

In [ ]:
plot_helper.plot_eff_fr_purity(eff_fr_colors)